In [ ]:
import stgen
import torch as th
model_name='t5-small'
t5_tokenizer=stgen.tokenizer(model_name)
text=['I love rock and roll']
x,y=t5_tokenizer.get_tokens_masks(text)
device= th.device('cuda')
batch_size=len(text)

In [ ]:
model=stgen.stgen(model_name,512,batch_size)
hidden=model.initHidden().to(device)
model=model.to(device)
output,hidden = model(x,y, hidden)